Imports

In [26]:
# Configuration file for the ML Food Buddy Recommender project
import os
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
import re

Functions

In [27]:
def data_loader(filename: str, source: str = "raw"):
    """
    Load a CSV dataset from the repo's data/raw or data/preprocessed folder.
    
    Parameters:
        filename: str - the CSV file name (can be compressed .zip or .gz)
        source: str - "raw" or "preprocessed" to select folder
    
    Returns:
        pd.DataFrame
    """
    if source not in ["raw", "preprocessed"]:
        raise ValueError("source must be 'raw' or 'preprocessed'")

    cwd = os.getcwd()
    repo_root = cwd

    # Walk upwards until we find the desired folder
    while True:
        data_path = os.path.join(repo_root, "data", source, filename)
        if os.path.exists(data_path):
            break
        parent = os.path.dirname(repo_root)
        if parent == repo_root:  # reached root of filesystem
            raise FileNotFoundError(f"Could not find {filename} in data/{source} from {cwd}")
        repo_root = parent

    # Detect compression type
    compression_type = None
    if filename.endswith('.zip'):
        compression_type = 'zip'
    elif filename.endswith('.gz'):
        compression_type = 'gzip'

    return pd.read_csv(data_path, compression=compression_type)


In [28]:
def parse_time(t):
    """
    Parse time in minutes. Supports:
    - Raw numeric strings (e.g., "45")
    - Already numeric values
    
    Returns:
        float (minutes) or np.nan if parsing fails
    """
    if pd.isna(t):
        return np.nan
    if isinstance(t, str):
        t = t.strip()
        if t.startswith("PT"):  # ISO 8601 duration
            hours = re.search(r'(\d+)H', t)
            minutes = re.search(r'(\d+)M', t)
            secs = re.search(r'(\d+)S', t)
            total_minutes = 0
            if hours:
                total_minutes += int(hours.group(1)) * 60
            if minutes:
                total_minutes += int(minutes.group(1))
            if secs:
                total_minutes += int(secs.group(1)) / 60
            return total_minutes if total_minutes > 0 else np.nan
        # fallback: try to parse as float
        try:
            return float(t)
        except:
            return np.nan
    # If already numeric
    try:
        return float(t)
    except:
        return np.nan


In [29]:
def format_time(t):
    if pd.isna(t):
        return None
    t = int(round(t))
    hours, minutes = divmod(t, 60)
    parts = []
    if hours > 0:
        parts.append(f"{hours} hour{'s' if hours > 1 else ''}")
    if minutes > 0:
        parts.append(f"{minutes} minute{'s' if minutes > 1 else ''}")
    return " ".join(parts) if parts else "0 minutes"

In [30]:
def clip_top_outliers(df, cols, z_thresh=3.5):
    """
    Clip only extreme outliers of selected columns using modified Z-score.
    
    Parameters:
    - df: pd.DataFrame
    - cols: list of str, columns to clip
    - z_thresh: float, threshold for modified Z-score (default=3.5)
    
    Returns:
    - df_clipped: pd.DataFrame with clipped values
    - thresholds: dict of column:clip_value for reference
    """
    df_clipped = df.copy()
    thresholds = {}
    
    for col in cols:
        if col not in df.columns:
            continue
        series = df[col]
        median = series.median()
        mad = np.median(np.abs(series - median))
        if mad == 0:
            continue  # can't detect outliers if MAD is zero
        mod_z = 0.6745 * (series - median) / mad
        upper_limit = series[mod_z <= z_thresh].max()  # largest non-outlier
        df_clipped[col] = np.minimum(series, upper_limit)
        thresholds[col] = upper_limit
    
    return df_clipped, thresholds

In [31]:
def parse_r_list_column(col):
    """
    Parse a column that looks like R-style list strings.

    - Fills NaNs with empty strings
    - Removes the c(...) wrapper
    - Strips out quotes
    """
    cleaned = col.fillna("").astype(str).str.strip()
    cleaned = cleaned.str.replace(r'^c\(|\)$', '', regex=True)  # remove c( and )
    cleaned = cleaned.str.replace(r'"', '', regex=False)       # remove quotes
    return cleaned


In [32]:
def clean_text(text):
    """
    Clean a text string by normalizing case, whitespace, and punctuation.

    - Converts to lowercase
    - Replaces multiple whitespace characters with a single space
    - Removes punctuation and special characters
    - Strips leading and trailing whitespace
    """
    text = str(text).lower()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text.strip()

EDA

In [33]:
# Load the dataset
recipes = data_loader("recipes.csv")
recipes.head()

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,39,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,NaN,"c(""Soak saffron in warm milk for 5 minutes and..."
2,40,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,NaN,"c(""Into a 1 quart Jar with tight fitting lid, ..."
3,41,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03T14:54:00Z,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"c(""Drain the tofu, carefully squeezing out exc..."
4,42,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19T06:19:00Z,Make and share this Cabbage Soup recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",...,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,NaN,"c(""Mix everything together and bring to a boil..."


In [34]:
# Info
recipes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522517 entries, 0 to 522516
Data columns (total 28 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   RecipeId                    522517 non-null  int64  
 1   Name                        522517 non-null  object 
 2   AuthorId                    522517 non-null  int64  
 3   AuthorName                  522517 non-null  object 
 4   CookTime                    439972 non-null  object 
 5   PrepTime                    522517 non-null  object 
 6   TotalTime                   522517 non-null  object 
 7   DatePublished               522517 non-null  object 
 8   Description                 522512 non-null  object 
 9   Images                      522516 non-null  object 
 10  RecipeCategory              521766 non-null  object 
 11  Keywords                    505280 non-null  object 
 12  RecipeIngredientQuantities  522514 non-null  object 
 13  RecipeIngredie

In [35]:
# Convert PrepTime into minutes
recipes['PrepTime_min'] = recipes['PrepTime'].apply(parse_time)

# Convert CookTime into minutes
recipes['CookTime_min'] = recipes['CookTime'].apply(parse_time)

# Convert TotalTime into minutes
recipes['TotalTime_min'] = recipes['TotalTime'].apply(parse_time)

In [36]:
# Identify unparseable PrepTime entries
invalid_prep_times = recipes[recipes['PrepTime'].apply(parse_time).isna()][['PrepTime']]
print("Invalid or unparseable PrepTime entries:")
print(invalid_prep_times.value_counts())

# Identify unparseable CookTime entries
invalid_cooking_times = recipes[recipes['CookTime'].apply(parse_time).isna()][['CookTime']]
print()
print("Invalid or unparseable CookTime entries:")
print(invalid_cooking_times.value_counts())

# Identify unparseable TotalTime entries
invalid_total_times = recipes[recipes['TotalTime'].apply(parse_time).isna()][['TotalTime']]
print()
print("Invalid or unparseable TotalTime entries:")
print(invalid_total_times.value_counts())

Invalid or unparseable PrepTime entries:
PrepTime
PT0S        15010
Name: count, dtype: int64

Invalid or unparseable CookTime entries:
Series([], Name: count, dtype: int64)

Invalid or unparseable TotalTime entries:
TotalTime
PT0S         2129
Name: count, dtype: int64


**Note:**  
The entries for Prep and Cook time identified as invalid or unparseable are likely incorrect or missing.  
It is recommended to replace these entries with the **median** or **average** value of the respective column to avoid skewing any analysis. 
Also, Let's check if they are affecting TotalTime.

In [37]:
# Compute difference
recipes['time_diff'] = recipes['PrepTime_min'] + recipes['CookTime_min'] - recipes['TotalTime_min']

# Find rows where the difference is not zero
mismatch = recipes[recipes['time_diff'].abs() != 0]

# Show relevant columns
mismatch.head()


,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions,PrepTime_min,CookTime_min,TotalTime_min,time_diff
8,46,A Jad - Cucumber Pickle,1533,Dancer,NaN,PT25M,PT25M,1999-08-11T19:48:00Z,Make and share this A Jad - Cucumber Pickle re...,character(0),...,0.2,0.2,0.1,NaN,1 cup,"c(""Slice the cucumber in four lengthwise, then...",25.0,NaN,25.0,NaN
10,48,Boston Cream Pie,1545,Nancy Van Ess,NaN,PT2H15M,PT2H15M,1999-08-24T04:35:00Z,Make and share this Boston Cream Pie recipe fr...,character(0),...,1.6,46.2,8.8,8.0,1 pie,"c(""Beat egg whites until soft peaks form."", ""G...",135.0,NaN,135.0,NaN
14,52,Cafe Cappuccino,2178,troyh,NaN,PT5M,PT5M,1999-08-31T21:05:00Z,Make and share this Cafe Cappuccino recipe fro...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,0.0,11.8,2.7,18.0,2 1/4 cups,"c(""Stir ingredients together."", ""Process in a ...",5.0,NaN,5.0,NaN
19,57,Black Bean Salsa,1569,Linda7,NaN,PT10M,PT10M,1999-08-31T21:02:00Z,Make and share this Black Bean Salsa recipe fr...,character(0),...,5.5,1.4,5.4,8.0,NaN,"c(""Combine all ingredients in a bowl."", ""Serve...",10.0,NaN,10.0,NaN
22,60,Blueberry Dessert,1545,Nancy Van Ess,NaN,PT35M,PT35M,1999-08-16T05:59:00Z,Make and share this Blueberry Dessert recipe f...,character(0),...,1.6,36.9,3.9,12.0,NaN,"c(""Heat oven to 400 degrees."", ""Mix 2 cups bak...",35.0,NaN,35.0,NaN


In [38]:
# Percentage of mismatches
mismatch.shape[0]/recipes.shape[0]

0.18157112591552046

In [39]:
# Fill missing prep/cook times with median
recipes['PrepTime_min'].fillna(recipes['PrepTime_min'].median(), inplace=True)
recipes['CookTime_min'].fillna(recipes['CookTime_min'].median(), inplace=True)

# Recalculate total time to ensure consistency
recipes['TotalTime_min'] = recipes['PrepTime_min'] + recipes['CookTime_min']

/var/folders/z0/2pzl523d7vb79mw5820jpx1r0000gn/T/ipykernel_5783/3988181521.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  recipes['PrepTime_min'].fillna(recipes['PrepTime_min'].median(), inplace=True)
/var/folders/z0/2pzl523d7vb79mw5820jpx1r0000gn/T/ipykernel_5783/3988181521.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on whic

In [40]:
# Convert DatePublished to datetime
recipes['DatePublished'] = pd.to_datetime(recipes['DatePublished'], errors='coerce')

In [41]:
# Remove timezone to make tz-naive
recipes['DatePublished'] = recipes['DatePublished'].dt.tz_convert(None)

# Identify unparseable dates
invalid_dates = recipes[recipes['DatePublished'].isna()][['DatePublished']]
print("Invalid or unparseable dates:")
print(invalid_dates.value_counts())

# Define a reasonable date range
today = pd.Timestamp.today()
min_reasonable_date = pd.Timestamp('1980-01-01')

# Identify dates outside reasonable range
unreasonable_dates = recipes[(recipes['DatePublished'] < min_reasonable_date) |
                             (recipes['DatePublished'] > today)][['DatePublished']]
print("Unreasonable dates:")
print(unreasonable_dates.value_counts())


Invalid or unparseable dates:
Series([], Name: count, dtype: int64)
Unreasonable dates:
Series([], Name: count, dtype: int64)


In [42]:
# Missing values percentage, excluding columns with 0% missing
missing_pct = recipes.isna().sum() / len(recipes) * 100
missing_pct = missing_pct[missing_pct > 0].sort_values(ascending=False)
print(missing_pct)

RecipeYield                   66.614292
AggregatedRating              48.462155
ReviewCount                   47.364775
RecipeServings                35.005751
time_diff                     18.152902
CookTime                      15.797572
Keywords                       3.298840
RecipeCategory                 0.143727
Description                    0.000957
RecipeIngredientQuantities     0.000574
Images                         0.000191
dtype: float64


In [43]:
# Show rows where Keywords is missing
recipes[recipes['Keywords'].isna()]

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions,PrepTime_min,CookTime_min,TotalTime_min,time_diff
25,63,Cabbage and Sausage Soup,1544,tranch,PT25M,PT15M,PT40M,1999-09-07 12:52:00,Make and share this Cabbage and Sausage Soup r...,character(0),...,3.5,11.6,30.2,6.0,NaN,"c(""In a medium stockpot or Dutch oven heat oli...",15.0,25.0,40.0,0.0
188,240,Chicken Fried Brown Rice,1572,Ed Paulhus,PT6M,PT10M,PT16M,1999-09-13 03:05:00,Make and share this Chicken Fried Brown Rice r...,"""https://img.sndimg.com/food/image/upload/w_55...",...,6.5,4.7,20.3,4.0,NaN,"c(""Heat large nonstick skillet over medium hea...",10.0,6.0,16.0,0.0
198,252,Sorrel Tarragon Sauce,1554,Jacques Lorrain,PT4H,PT25M,PT4H25M,1999-10-03 23:28:00,Make and share this Sorrel Tarragon Sauce reci...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,0.7,11.9,5.3,NaN,1 1/2 cups,"c(""Mix all ingredients in medium bowl."", ""Seas...",25.0,240.0,265.0,0.0
493,580,Garlic Mushroom Sauce,1543,Doreen Randal,PT6M,PT30M,PT36M,1999-08-27 05:13:00,Make and share this Garlic Mushroom Sauce reci...,character(0),...,0.5,1.1,1.8,4.0,NaN,"c(""Crushed garlic, peel and chop finely."", ""Wi...",30.0,6.0,36.0,0.0
591,680,Creamy Smoked Salmon & Dijon Pasta,1556,Strawberry Girl,PT5M,PT45M,PT50M,1999-09-06 04:24:00,Make and share this Creamy Smoked Salmon & Dij...,character(0),...,2.4,3.0,20.1,4.0,NaN,"c(""Cook the pasta."", ""Drain but do not rinse. ...",45.0,5.0,50.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522467,541334,Easy Tater Tot Hotdish,274666,Wendelina,PT55M,PT15M,PT1H10M,2020-12-08 19:58:00,Make and share this Easy Tater Tot Hotdish rec...,character(0),...,6.7,2.1,38.0,6.0,1 9x13 pan,"c(""In a large saucepan, mix ground beef, onion...",15.0,55.0,70.0,0.0
522470,541337,Firehouse Favorite Casserole,2001361961,Dori K.,PT55M,PT15M,PT1H10M,2020-12-15 19:17:00,Originally I got this recipe off of the Grandm...,character(0),...,5.8,11.6,44.7,6.0,NaN,"c(""Cook frozen egg noodles according to packag...",15.0,55.0,70.0,0.0
522476,541343,Shrimp Cocktail Bar,2001112113,Jonathan Melendez,PT45M,PT20M,PT1H5M,2020-12-16 18:40:00,Whether you're ringing in the New Year or just...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,8.5,11.9,45.9,NaN,NaN,"c(""You can decide to go either the classic rou...",20.0,45.0,65.0,0.0
522486,541353,Jewish-Style Braised Beef Brisket,181957,Oliver1010,PT4H,PT45M,PT4H45M,2020-12-21 15:50:00,This is a classic &quot;Jewish&quot; style bri...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,4.1,8.2,72.7,8.0,NaN,"c(""Preheat oven to 300&deg;F Season brisket a...",45.0,240.0,285.0,0.0


In [44]:
# Display summary statistics for all
print(recipes.describe())

            RecipeId      AuthorId                  DatePublished  \
count  522517.000000  5.225170e+05                         522517   
mean   271821.436970  4.572585e+07  2008-01-18 06:34:47.071884800   
min        38.000000  2.700000e+01            1999-08-06 00:40:00   
25%    137206.000000  6.947400e+04            2005-09-13 10:25:00   
50%    271758.000000  2.389370e+05            2007-12-13 16:27:00   
75%    406145.000000  5.658280e+05            2009-12-31 09:45:00   
max    541383.000000  2.002886e+09            2020-12-22 22:12:00   
std    155495.878422  2.929714e+08                            NaN   

       AggregatedRating    ReviewCount       Calories     FatContent  \
count     269294.000000  275028.000000  522517.000000  522517.000000   
mean           4.632014       5.227784     484.438580      24.614922   
min            1.000000       1.000000       0.000000       0.000000   
25%            4.500000       1.000000     174.200000       5.600000   
50%            5.0

**Dataset Summary Observations:**

- **Extreme scales & outliers:** `Calories`, `FatContent`, `SodiumContent`, `SugarContent`, `RecipeServings` max far above 75th percentile – likely errors.  
- **Skewed distributions:** Means > medians for nutrition columns – right-skewed.  
- **Missing data:** `AggregatedRating`, `ReviewCount`, `RecipeServings` have many NaNs.  
- **Time inconsistencies:** `PrepTime_min`, `CookTime_min`, `TotalTime_min` have unrealistic maxima

Preprocessing / Cleaning

In [50]:
# Columns to clip
numeric_cols_to_clip = [
    "TotalTime_min", "Calories", "FatContent", "SaturatedFatContent",
    "CholesterolContent", "SodiumContent", "CarbohydrateContent",
    "FiberContent", "SugarContent", "ProteinContent"
]

# Apply clipping
recipes_clipped, clip_thresholds = clip_top_outliers(recipes, numeric_cols_to_clip)

# Show thresholds
for col, val in clip_thresholds.items():
    print(f"{col} clipped at: {val:.2f}")


TotalTime_min clipped at: 122.00
Calories clipped at: 1173.20
FatContent clipped at: 64.10
SaturatedFatContent clipped at: 24.40
CholesterolContent clipped at: 263.60
SodiumContent clipped at: 1792.20
CarbohydrateContent clipped at: 120.00
FiberContent clipped at: 10.50
SugarContent clipped at: 32.80
ProteinContent clipped at: 46.90


In [46]:
# Parse and clean string columns
recipes['ingredients_clean'] = parse_r_list_column(recipes['RecipeIngredientParts'])
recipes['category_clean'] = parse_r_list_column(recipes['RecipeCategory']) if 'RecipeCategory' in recipes.columns else ""
recipes['keywords_clean'] = parse_r_list_column(recipes['Keywords']) if 'Keywords' in recipes.columns else ""

In [47]:
# Apply cleaning to all relevant columns
recipes['combined_text'] = (
    recipes['ingredients_clean'] + ", " +
    recipes['category_clean'] + ", " +
    recipes['keywords_clean'] + ", " +
    recipes['Description'].fillna("")
).apply(clean_text)

In [48]:
recipes.head()

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,RecipeYield,RecipeInstructions,PrepTime_min,CookTime_min,TotalTime_min,time_diff,ingredients_clean,category_clean,keywords_clean,combined_text
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09 21:46:00,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan...",45.0,1440.0,1485.0,0.0,"blueberries, granulated sugar, vanilla yogurt,...",Frozen Desserts,"Dessert, Low Protein, Low Cholesterol, Healthy...",blueberries granulated sugar vanilla yogurt le...
1,39,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29 13:12:00,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",...,NaN,"c(""Soak saffron in warm milk for 5 minutes and...",240.0,25.0,265.0,0.0,"saffron, milk, hot green chili peppers, onions...",Chicken Breast,"Chicken Thigh & Leg, Chicken, Poultry, Meat, A...",saffron milk hot green chili peppers onions ga...
2,40,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05 19:52:00,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,NaN,"c(""Into a 1 quart Jar with tight fitting lid, ...",30.0,5.0,35.0,0.0,"sugar, lemons, rind of, lemon, zest of, fresh ...",Beverages,"Low Protein, Low Cholesterol, Healthy, Summer,...",sugar lemons rind of lemon zest of fresh water...
3,41,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03 14:54:00,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,4 kebabs,"c(""Drain the tofu, carefully squeezing out exc...",1440.0,20.0,1460.0,0.0,"extra firm tofu, eggplant, zucchini, mushrooms...",Soy/Tofu,"Beans, Vegetable, Low Cholesterol, Weeknight, ...",extra firm tofu eggplant zucchini mushrooms so...
4,42,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19 06:19:00,Make and share this Cabbage Soup recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",...,NaN,"c(""Mix everything together and bring to a boil...",20.0,30.0,50.0,0.0,"plain tomato juice, cabbage, onion, carrots, c...",Vegetable,"Low Protein, Vegan, Low Cholesterol, Healthy, ...",plain tomato juice cabbage onion carrots celer...
